# 🚀 F2 Portfolio Recommender Agent - Demo Notebook

## Autonomous AI-Powered Portfolio Optimization for 2025

**Hackathon Project**: GenAI Agentic AI Solution

**Value Proposition**: Democratizing personalized, transparent, and compliant portfolio advice through agentic AI

---

### What This Demo Showcases

1. **Multi-Step Agentic Reasoning**: Tool selection, parameter extraction, and result synthesis
2. **Quantitative Finance Integration**: Real portfolio optimization using Modern Portfolio Theory
3. **Safety Guardrails**: PII detection, output validation, mandatory disclaimers
4. **Explainability**: Plain-English explanations of complex quantitative recommendations
5. **Evaluation**: Agentic metrics beyond traditional accuracy

---

## 📦 Setup & Dependencies

In [ ]:
# Install dependencies (run once)
# !pip install -r requirements.txt -q

In [ ]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import json
from pprint import pprint

# Add project root to path
if os.path.abspath('..') not in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

# Project imports
from portfolio_optimizer import portfolio_optimizer_tool, PortfolioOptimizer
from guardrails import check_input_safety, apply_output_guardrails, GuardrailSystem
from config import STOCK_UNIVERSE, RISK_PROFILES, TEST_CASES

print("✅ Imports successful!")

In [ ]:
# Configuration check
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY:
    print("⚠️ WARNING: OPENAI_API_KEY not set!")
    print("   Agent features will be limited.")
    print("   Set your API key in .env file to enable full functionality.")
else:
    print(f"✅ OpenAI API Key configured (length: {len(OPENAI_API_KEY)})")

---

## 🧪 Part 1: Portfolio Optimization Tool (Quantitative Core)

Testing the quantitative engine powered by PyPortfolioOpt

In [ ]:
# Test Case 1: Conservative Long-Term Investor
print("="*70)
print("Test Case 1: Conservative Investor, 10-Year Horizon")
print("="*70)

result_conservative = portfolio_optimizer_tool(
    risk_profile='low',
    horizon_years=10
)

if 'error' not in result_conservative or not result_conservative['error']:
    print("\n📊 Portfolio Allocation:")
    for ticker, weight in result_conservative['portfolio_weights'].items():
        print(f"   {ticker}: {weight*100:.2f}%")
    
    print("\n📈 Expected Performance:")
    metrics = result_conservative['metrics']
    print(f"   Annual Return: {metrics['expected_annual_return']*100:.2f}%")
    print(f"   Volatility: {metrics['annual_volatility']*100:.2f}%")
    print(f"   Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
    
    print("\n💡 Explanation:")
    print(result_conservative['explanation'])
else:
    print(f"❌ Error: {result_conservative['error']}")

In [ ]:
# Test Case 2: Moderate Investor
print("="*70)
print("Test Case 2: Moderate Investor, 5-Year Horizon")
print("="*70)

result_moderate = portfolio_optimizer_tool(
    risk_profile='medium',
    horizon_years=5
)

if 'portfolio_weights' in result_moderate:
    print("\n📊 Portfolio Allocation:")
    for ticker, weight in sorted(result_moderate['portfolio_weights'].items(), 
                                   key=lambda x: x[1], reverse=True):
        print(f"   {ticker}: {weight*100:.2f}%")
    
    print(f"\n📈 Expected Return: {result_moderate['metrics']['expected_annual_return']*100:.2f}%")
    print(f"📉 Risk (Volatility): {result_moderate['metrics']['annual_volatility']*100:.2f}%")

In [ ]:
# Test Case 3: Aggressive Investor
print("="*70)
print("Test Case 3: Aggressive Investor, 3-Year Horizon")
print("="*70)

result_aggressive = portfolio_optimizer_tool(
    risk_profile='high',
    horizon_years=3
)

if 'portfolio_weights' in result_aggressive:
    print("\n📊 Top Holdings:")
    holdings = sorted(result_aggressive['portfolio_weights'].items(), 
                      key=lambda x: x[1], reverse=True)[:5]
    for ticker, weight in holdings:
        print(f"   {ticker}: {weight*100:.2f}%")
    
    print(f"\n⚡ Sharpe Ratio: {result_aggressive['metrics']['sharpe_ratio']:.2f}")
    print(f"📊 Diversification: {result_aggressive['metadata']['num_assets']} assets")

### 📊 Visual Comparison of Risk Profiles

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Collect results
profiles = ['Low Risk\n(10y)', 'Medium Risk\n(5y)', 'High Risk\n(3y)']
results = [result_conservative, result_moderate, result_aggressive]

returns = [r['metrics']['expected_annual_return']*100 for r in results if 'metrics' in r]
volatilities = [r['metrics']['annual_volatility']*100 for r in results if 'metrics' in r]
sharpes = [r['metrics']['sharpe_ratio'] for r in results if 'metrics' in r]

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Expected Returns
axes[0].bar(profiles[:len(returns)], returns, color=['green', 'orange', 'red'])
axes[0].set_title('Expected Annual Returns', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Return (%)')
axes[0].grid(axis='y', alpha=0.3)

# Volatility
axes[1].bar(profiles[:len(volatilities)], volatilities, color=['green', 'orange', 'red'])
axes[1].set_title('Annual Volatility (Risk)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Volatility (%)')
axes[1].grid(axis='y', alpha=0.3)

# Sharpe Ratios
axes[2].bar(profiles[:len(sharpes)], sharpes, color=['green', 'orange', 'red'])
axes[2].set_title('Sharpe Ratio (Risk-Adjusted Return)', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Sharpe Ratio')
axes[2].grid(axis='y', alpha=0.3)
axes[2].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.savefig('portfolio_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualization saved as 'portfolio_comparison.png'")

---

## 🛡️ Part 2: Safety Guardrails Testing

Demonstrating responsible AI: PII detection, input validation, output compliance

In [ ]:
# Test PII Detection
print("="*70)
print("Guardrail Test 1: PII Detection")
print("="*70)

test_inputs = [
    ("Safe: Medium risk for 5 years", True),
    ("Unsafe: My PAN is ABCDE1234F, recommend portfolio", False),
    ("Unsafe: SSN 123-45-6789", False),
    ("Unsafe: My Aadhaar number is 123456789012", False),
]

for test_input, expected_safe in test_inputs:
    is_safe, error = check_input_safety(test_input)
    status = "✅ PASS" if (is_safe == expected_safe) else "❌ FAIL"
    safety_status = "SAFE" if is_safe else "BLOCKED"
    
    print(f"\n{status} [{safety_status}]: '{test_input[:50]}...'")
    if error:
        print(f"   Reason: {error}")

In [ ]:
# Test Disclaimer Enforcement
print("\n" + "="*70)
print("Guardrail Test 2: Disclaimer Enforcement")
print("="*70)

# Create a portfolio result without disclaimer
sample_portfolio = {
    'portfolio_weights': {'RELIANCE.NS': 0.5, 'TCS.NS': 0.5},
    'metrics': {
        'expected_annual_return': 0.15,
        'annual_volatility': 0.20
    },
    'explanation': 'This is a balanced portfolio.'
}

# Apply guardrails
guardrailed_output = apply_output_guardrails(sample_portfolio)

print("\n📝 Original Explanation:")
print(sample_portfolio['explanation'])

print("\n🛡️ Guardrailed Explanation:")
print(guardrailed_output['explanation'])

print("\n✅ Guardrails Applied:")
pprint(guardrailed_output['guardrails_applied'])

In [ ]:
# Test Output Validation
print("\n" + "="*70)
print("Guardrail Test 3: Output Validation")
print("="*70)

guardrail = GuardrailSystem()

# Test valid output
valid_output = {
    'portfolio_weights': {'A': 0.5, 'B': 0.5},
    'metrics': {
        'expected_annual_return': 0.12,
        'annual_volatility': 0.18
    }
}

validation = guardrail.validate_portfolio_output(valid_output)
print(f"\nValid Output Test: {'✅ PASS' if validation['is_valid'] else '❌ FAIL'}")

# Test invalid output (weights don't sum to 1)
invalid_output = {
    'portfolio_weights': {'A': 0.3, 'B': 0.3},  # Sum = 0.6, should be ~1.0
    'metrics': {'expected_annual_return': 0.12}
}

validation_invalid = guardrail.validate_portfolio_output(invalid_output)
print(f"\nInvalid Output Test: {'❌ Correctly Rejected' if not validation_invalid['is_valid'] else '✅ Incorrectly Accepted'}")
if validation_invalid['violations']:
    print("\n⚠️ Violations Detected:")
    for v in validation_invalid['violations']:
        print(f"   - {v['type']}: {v.get('message', 'N/A')}")

---

## 🤖 Part 3: Agentic AI System (Full Pipeline)

**Note**: This section requires a valid OpenAI API key. If you don't have one set, the cells below will be skipped.

In [ ]:
# Check if agent can be initialized
agent_available = bool(OPENAI_API_KEY)

if agent_available:
    from agent import PortfolioRecommenderAgent, get_portfolio_recommendation
    print("✅ Agent module loaded successfully!")
    print("   Ready for agentic demonstrations.")
else:
    print("⚠️ Agent features disabled (no API key)")
    print("   Skipping agentic AI demonstrations.")
    print("   Set OPENAI_API_KEY in .env to enable.")

In [ ]:
# Agent Test 1: Natural Language Query Processing
if agent_available:
    print("="*70)
    print("Agent Test 1: Natural Language Understanding")
    print("="*70)
    
    query = "I'm 30 years old, moderately risk-tolerant, and saving for a house down payment in 7 years."
    
    print(f"\n📝 User Query:\n   '{query}'")
    print("\n🤖 Agent Processing...\n")
    
    response = get_portfolio_recommendation(query, verbose=True)
    
    if response.get('success'):
        print("\n✅ Agent Response:")
        print("\n📊 Portfolio:")
        pprint(response['portfolio'])
        
        print("\n📈 Metrics:")
        pprint(response['metrics'])
        
        print("\n💡 Explanation:")
        print(response['explanation'][:500] + "...")
        
        print("\n🔧 Agent Metadata:")
        pprint(response.get('agent_metadata', {}))
    else:
        print(f"\n❌ Agent failed: {response.get('error')}")
        print(f"   Message: {response.get('message')}")
else:
    print("⏭️ Skipped: Agent requires API key")

In [ ]:
# Agent Test 2: Edge Case - Ambiguous Query
if agent_available:
    print("\n" + "="*70)
    print("Agent Test 2: Handling Ambiguous Input")
    print("="*70)
    
    ambiguous_query = "I want to invest but don't know much about risk."
    
    print(f"\n📝 User Query:\n   '{ambiguous_query}'")
    print("\n🤖 Agent Processing...\n")
    
    response = get_portfolio_recommendation(ambiguous_query, verbose=False)
    
    if response.get('success'):
        print("✅ Agent handled ambiguity successfully!")
        print(f"\n📊 Inferred Risk: {response['metrics'].get('risk_profile', 'N/A')}")
        print(f"📅 Inferred Horizon: {response['metrics'].get('horizon_years', 'N/A')} years")
    else:
        print(f"Agent response: {response.get('message', 'No message')}")
else:
    print("⏭️ Skipped: Agent requires API key")

---

## 📊 Part 4: Agentic Evaluation Metrics

Demonstrating advanced evaluation beyond traditional accuracy

In [ ]:
from tests.test_agent import AgenticEvaluationMetrics

metrics = AgenticEvaluationMetrics()

print("="*70)
print("Agentic Evaluation Framework")
print("="*70)

# Evaluate a sample response
sample_response = {
    'portfolio_weights': {'RELIANCE.NS': 0.3, 'TCS.NS': 0.4, 'INFY.NS': 0.3},
    'metrics': {
        'expected_annual_return': 0.15,
        'annual_volatility': 0.22,
        'risk_profile': 'medium'
    },
    'explanation': '''This portfolio is optimized for medium risk tolerance with a 5-year horizon.
    Expected annual return is 15% with 22% volatility.
    Diversification across technology leaders provides balanced growth.
    DISCLAIMER: This is not financial advice and is for demonstration purposes only.''',
    'guardrails_applied': {'disclaimer_enforced': True},
    'success': True
}

# Calculate metrics
format_score = metrics.output_format_validity(sample_response)
explanation_score = metrics.explanation_quality(sample_response['explanation'])
guardrail_score = metrics.guardrail_compliance(sample_response)

print("\n📊 Evaluation Scores:")
print(f"   • Output Format Validity: {format_score:.2f} / 1.00")
print(f"   • Explanation Quality: {explanation_score:.2f} / 1.00")
print(f"   • Guardrail Compliance: {guardrail_score:.2f} / 1.00")
print(f"\n   📈 Overall Score: {(format_score + explanation_score + guardrail_score) / 3:.2f} / 1.00")

# Grading
overall = (format_score + explanation_score + guardrail_score) / 3
grade = 'A' if overall >= 0.8 else 'B' if overall >= 0.6 else 'C'
print(f"\n   🏆 Grade: {grade}")

In [ ]:
# Test Argument Correctness Metric
print("\n" + "="*70)
print("Testing Argument Correctness Metric")
print("="*70)

test_cases = [
    {
        "name": "Perfect Match",
        "expected": {"risk_profile": "medium", "horizon_years": 5},
        "actual": {"risk_profile": "medium", "horizon_years": 5}
    },
    {
        "name": "Partial Match (Risk Correct)",
        "expected": {"risk_profile": "medium", "horizon_years": 5},
        "actual": {"risk_profile": "medium", "horizon_years": 7}
    },
    {
        "name": "No Match",
        "expected": {"risk_profile": "low", "horizon_years": 10},
        "actual": {"risk_profile": "high", "horizon_years": 3}
    }
]

for test in test_cases:
    score = metrics.argument_correctness(test["expected"], test["actual"])
    print(f"\n{test['name']}: {score:.2f}")
    print(f"   Expected: {test['expected']}")
    print(f"   Actual: {test['actual']}")

---

## 🎯 Part 5: Hackathon Demo Scenarios

Real-world use cases demonstrating the agent's value

In [ ]:
# Scenario 1: First-time Investor (Young Professional)
print("="*70)
print("Scenario 1: First-Time Investor - Young Professional")
print("="*70)
print("\n👤 Profile: 25-year-old software engineer, new to investing")
print("🎯 Goal: Build wealth for future, can tolerate moderate risk")
print("⏱️ Timeline: 10+ years")

result = portfolio_optimizer_tool(risk_profile='medium', horizon_years=10)

if 'portfolio_weights' in result:
    print("\n🤖 Agent Recommendation:")
    print(result['explanation'][:400] + "...\n")
    
    print("📊 Portfolio Breakdown:")
    for ticker, weight in sorted(result['portfolio_weights'].items(), 
                                   key=lambda x: x[1], reverse=True)[:3]:
        print(f"   {ticker}: {weight*100:.1f}%")
    
    print(f"\n✅ Expected to grow ₹100,000 to ~₹{100000 * (1 + result['metrics']['expected_annual_return'])**10:,.0f} in 10 years")

In [ ]:
# Scenario 2: Pre-Retiree (Conservative Focus)
print("\n" + "="*70)
print("Scenario 2: Pre-Retiree - Capital Preservation")
print("="*70)
print("\n👤 Profile: 55-year-old planning retirement in 5 years")
print("🎯 Goal: Preserve capital, generate steady income")
print("⏱️ Timeline: 5 years")

result = portfolio_optimizer_tool(risk_profile='low', horizon_years=5)

if 'portfolio_weights' in result:
    print("\n🤖 Agent Recommendation:")
    print("   ✅ Conservative allocation focusing on stable, established companies")
    print(f"   📉 Lower volatility: {result['metrics']['annual_volatility']*100:.1f}%")
    print(f"   📈 Steady returns: {result['metrics']['expected_annual_return']*100:.1f}% annually")
    print(f"   ⚡ Sharpe Ratio: {result['metrics']['sharpe_ratio']:.2f} (risk-adjusted efficiency)")

In [ ]:
# Scenario 3: Entrepreneur (High Risk Tolerance)
print("\n" + "="*70)
print("Scenario 3: Entrepreneur - Aggressive Growth")
print("="*70)
print("\n👤 Profile: 35-year-old business owner, high risk appetite")
print("🎯 Goal: Maximize growth, willing to accept volatility")
print("⏱️ Timeline: 7 years")

result = portfolio_optimizer_tool(risk_profile='high', horizon_years=7)

if 'portfolio_weights' in result:
    print("\n🤖 Agent Recommendation:")
    print("   ✅ Aggressive allocation with high-growth potential")
    print(f"   📈 Higher expected returns: {result['metrics']['expected_annual_return']*100:.1f}%")
    print(f"   📊 Diversified across {result['metadata']['num_assets']} growth stocks")
    
    print("\n💰 Growth Projection (₹500,000 initial):")
    ret = result['metrics']['expected_annual_return']
    for year in [3, 5, 7]:
        value = 500000 * (1 + ret) ** year
        print(f"   Year {year}: ₹{value:,.0f}")

---

## 📋 Summary & Key Takeaways

In [ ]:
print("="*70)
print("🏆 F2 Portfolio Recommender Agent - Hackathon Summary")
print("="*70)

summary = """
✅ DEMONSTRATED CAPABILITIES:

1️⃣ AGENTIC REASONING
   • Multi-step tool orchestration
   • Natural language parameter extraction
   • Context-aware decision making

2️⃣ QUANTITATIVE FINANCE INTEGRATION
   • Real portfolio optimization using Modern Portfolio Theory
   • Risk-adjusted allocations (Sharpe ratio optimization)
   • Dynamic adaptation to user profiles

3️⃣ SAFETY & COMPLIANCE
   • PII detection (PAN, Aadhaar, SSN)
   • Mandatory disclaimer enforcement
   • Output validation (weights, metrics bounds)

4️⃣ EXPLAINABILITY
   • Plain-English portfolio explanations
   • Risk rationale and diversification logic
   • Transparent decision process

5️⃣ EVALUATION FRAMEWORK
   • Agentic metrics (beyond accuracy)
   • Tool correctness & argument accuracy
   • Reproducible testing

🎯 REAL-WORLD IMPACT:
   • Democratizes personalized financial advice
   • Scalable to millions of retail investors
   • Regulatory-ready (SEBI compliance considerations)
   • Production-feasible architecture

📊 TECHNICAL EXCELLENCE:
   • Modern tech stack (GPT-4o, LangChain, PyPortfolioOpt)
   • Modular, maintainable code
   • Comprehensive documentation
   • 12-hour MVP feasibility

🚀 NEXT STEPS FOR PRODUCTION:
   • Add more asset classes (bonds, ETFs, mutual funds)
   • User authentication & portfolio tracking
   • Regulatory compliance certification
   • A/B testing & continuous learning
"""

print(summary)

print("\n" + "="*70)
print("Thank you for exploring the F2 Portfolio Recommender Agent!")
print("="*70)

---

## 📚 References & Resources

1. **Modern Portfolio Theory**: Markowitz (1952) - Portfolio Selection
2. **Agentic AI**: LangChain Documentation - Multi-Agent Systems
3. **PyPortfolioOpt**: [GitHub Repository](https://github.com/robertmartin8/PyPortfolioOpt)
4. **SEBI Guidelines**: Securities and Exchange Board of India - Robo-Advisory
5. **GenAI Best Practices**: OpenAI - Responsible AI Guidelines

---

**License**: MIT License (for hackathon/educational purposes)

**Disclaimer**: This project is a demonstration of agentic AI capabilities and is NOT intended for actual financial decision-making. Always consult qualified financial advisors for investment advice.